In [ ]:
# 패키지 수입
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import InputLayer, Conv2D, MaxPool2D, Flatten, Dense

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [ ]:
# 하이퍼 패러미터 설정
MY_EPOCH = 50
MY_BATCH = 100

In [ ]:
# 데이터 불러오기
digits = load_digits()
X = digits.data
Y = digits.target

In [ ]:
# 샘플 데이터 시각화
plt.rcParams["figure.figsize"] = (10, 10)
sample_num = 100

fig, ax = plt.subplots(3, 3)

for row in range(3):
    for col in range(3):
        sub = ax[row, col]
        sub.imshow(X[sample_num].reshape(8, 8), cmap="gray")

        sub.set_title(str(sample_num) + ": " + str(Y[sample_num]))
        sub.set_xticks([])
        sub.set_yticks([])

        sample_num += 100

plt.show()

In [ ]:
# 입력 데이터에 색상 채널 수 추가
X = X.reshape(X.shape[0], 8, 8, 1)

In [ ]:
# 입력 데이터 스케일링
X = X / 255.0

In [ ]:
# 출력 데이터 원-핫 인코딩
Y = np_utils.to_categorical(Y, 10)

In [ ]:
# 학습용, 평가용 데이터 분할
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=43)

In [ ]:
# 인공신경망 구축
model = Sequential()

model.add(InputLayer(input_shape=(8, 8, 1)))

model.add(Conv2D(12, kernel_size=(4, 4), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2, 2), padding="same"))

model.add(Conv2D(24, kernel_size=(4, 4), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2, 2), padding="same"))

model.add(Flatten())
model.add(Dense(2000, activation="relu"))
model.add(Dense(10, activation="softmax"))

model.summary()

In [ ]:
# 기타 옵션 설정
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["acc"])

In [ ]:
# 학습 진행
hist = model.fit(X_train, Y_train, batch_size=MY_BATCH, epochs=MY_EPOCH, verbose=1, validation_data=(X_test, Y_test))

In [ ]:
# 학습 내역 시각화
fig, loss_ax = plt.subplots()
acc_ax = loss_ax.twinx()

loss_ax.plot(hist.history["loss"], "r", label="train loss")
loss_ax.plot(hist.history["val_loss"], "m", label="val loss")

acc_ax.plot(hist.history["acc"], "b", label="train acc")
acc_ax.plot(hist.history["val_acc"], "c", label="val acc")

loss_ax.set_xlabel("Epochs")
loss_ax.set_ylabel("Loss")
acc_ax.set_ylabel("Accuracy")

loss_ax.legend()
acc_ax.legend()

plt.show()

In [ ]:
# 학습 모델 평가
loss, acc = model.evaluate(X_test, Y_test, batch_size=MY_BATCH, verbose=1)
print("Test loss:", loss)
print("Test accuracy:", acc)

In [ ]:
# 예측 데이터 생성
Y_pred = model.predict(X_test, batch_size=MY_BATCH, verbose=1)

In [ ]:
# 원-핫 인코딩 되돌리기
Y_test = np.argmax(Y_test, axis=1)
Y_pred = np.argmax(Y_pred, axis=1)

In [ ]:
# 혼동 행렬 생성
cm = confusion_matrix(Y_test, Y_pred)
cm = pd.DataFrame(cm)
display(cm)

In [ ]:
# 틀린 예측 데이터 시각화
row = 3
col = 3

fig, ax = plt.subplots(row, col)

i = 0
cnt = 0

while cnt < (row * col):
    if Y_test[i] == Y_pred[i]:
        i += 1
        continue

    sub = ax[int(cnt/row), int(cnt%col)]
    sub.imshow(X_test[i].reshape(8, 8), cmap="gray")

    title = str(i) + "- Predict: " + str(Y_pred[i]) + "\nActual: " + str(Y_test[i]) 
    sub.set_title(title)
    sub.set_xticks([])
    sub.set_yticks([])

    i += 1
    cnt += 1

plt.show()